## Test for testing baseline methods
### Supervised: SVM, RF, XGBOOST
### Semi-supervised: TSVM

In [1]:
import numpy as np
import visdom
import utils
import seaborn as sns
from datasets import get_patch_data, get_pixel_idx, HyperX_patches
import os
from sklearn import preprocessing
from sklearn.decomposition import PCA
import random
import math
from matplotlib import pyplot as plt
import torch.utils.data as data

In [53]:
vis = visdom.Visdom()

Setting up a new session...


In [2]:
data_path = '/home/oscar/Desktop/Exjobb/Data/ieee_supplement/Hyperspectral_Grids/Salinas/'

label_values = ["Undefined", "Brocoli_green_weeds_1", "Brocoli_green_weeds_2", "Fallow",
                        "Fallow_rough_plow", "Fallow_smooth", "Stubble",
                        "Celery", "Grapes_untrained", "Soil_vinyard_develop",
                        "Corn_senesced_green_weeds", "Lettuce_romaine_4wk", "Lettuce_romaine_5wk",
                        "Lettuce_romaine_6wk", "Lettuce_romaine_7wk", "Vinyard_untrained",
                        "Vinyard_vertical_trellis"]

palette = None
rgb_bands = (43, 21, 11)

if palette is None:
    # Generate color palette
    palette = {0: (0, 0, 0)}
    for k, color in enumerate(sns.color_palette("hls", len(label_values) - 1)):
        palette[k + 1] = tuple(np.asarray(255 * np.array(color), dtype='uint8'))
invert_palette = {v: k for k, v in palette.items()}

def convert_to_color(x):
    return utils.convert_to_color_(x, palette=palette)
def convert_from_color(x):
    return utils.convert_from_color_(x, palette=invert_palette)

In [3]:
fold = 0

train_img, train_gt, test_img, test_gt, label_values, ignored_labels, rgb_bands, palette = get_patch_data('Salinas', 5, target_folder=data_path, fold=fold)

n_bands = train_img.shape[-1]
n_classes = len(label_values) - len(ignored_labels)

idx_sup, idx_val, idx_unsup = get_pixel_idx(train_img, train_gt, ignored_labels, 5)

hyperparams = {'patch_size': 5, 'dataset': 'Salinas', 'ignored_labels': ignored_labels, 
               'flip_augmentation': False, 'radiation_augmentation': False, 'mixture_augmentation': False,
              'center_pixel': True, 'supervision': 'full', 'pca_augmentation': False, 'pca_strength': 1, 'cutout_spatial': False,
              'cutout_spectral': False, 'augmentation_magnitude': 1, 'spatial_combinations': False, 'spectral_mean': False,
              'moving_average': False}

val_dataset = HyperX_patches(train_img, train_gt, idx_val, labeled=True, **hyperparams)
val_loader = data.DataLoader(val_dataset, batch_size=10)

train_labeled_dataset = HyperX_patches(train_img, train_gt, idx_sup, labeled=True, **hyperparams)
train_labeled_loader = data.DataLoader(train_labeled_dataset, batch_size=len(idx_sup),
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)

unlabeled_ratio = math.ceil(len(idx_unsup)/len(idx_sup))

train_unlabeled_dataset = HyperX_patches(train_img, train_gt, idx_unsup, labeled=False, **hyperparams)
train_unlabeled_loader = data.DataLoader(train_unlabeled_dataset, batch_size=len(idx_unsup),
                                       #pin_memory=True, num_workers=5,
                                       shuffle=True, drop_last=True)
amount_labeled = idx_sup.shape[0]

Move data to simple list with pixel form

In [4]:
i = 0
X = np.zeros((len(idx_sup), n_bands))
Y = np.zeros(len(idx_sup))
for p, x, y in idx_sup:
    X[i, :] = train_img[p,x,y]
    Y[i] = train_gt[p,x,y] - 1
    i += 1

In [5]:
i = 0
X_un = np.zeros((len(idx_unsup), n_bands))
for p, x, y in idx_unsup:
    X_un[i, :] = train_img[p,x,y]
    i += 1

### SVM

In [83]:
import sklearn.svm as SVM

In [84]:
SVM_GRID_PARAMS = [{'kernel': ['rbf'], 'gamma': [5e-1, 2e-1], 'C': [1000, 1500]}]

class_weight = None
clf = SVM.SVC(class_weight=class_weight)
clf = sklearn.model_selection.GridSearchCV(clf, SVM_GRID_PARAMS, verbose=5, n_jobs=4)
clf.fit(X, Y)
print('SVMs best params: {} '.format(clf.best_params_))
#save_model(clf, 'SVM', 'Salinas')
prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

/home/oscar/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=4)]: Done   8 out of  12 | elapsed:    5.4s remaining:    2.7s
[Parallel(n_jobs=4)]: Done  12 out of  12 | elapsed:    8.0s finished


SVMs best params: {'C': 1000, 'gamma': 0.5, 'kernel': 'rbf'} 


### Random Forest

In [86]:
from sklearn.ensemble import RandomForestClassifier
import sklearn

In [87]:
#RF_GRID_PARAMS = [{'max_depth': [32, 256, 512], 'n_estimators': [100, 200, 400]}]

clf = RandomForestClassifier(n_estimators = 200, min_samples_split=2, \
                    max_features=10, max_depth=10)
#clf = sklearn.model_selection.GridSearchCV(clf, RF_GRID_PARAMS, verbose=5, n_jobs=4)
clf.fit(X, Y)
#print('RFs best params: {} '.format(clf.best_params_))

prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

### XGBoost

In [90]:
from xgboost import XGBClassifier

In [91]:
clf = XGBClassifier()
clf.fit(X, Y)

prediction = clf.predict(test_img.reshape(-1, n_bands))
prediction = prediction.reshape(test_img.shape[:2])

### TSVM

In [6]:
from scipy.optimize import minimize

Initialize parameters

In [35]:
nb_labeled = len(X)
nb_unlabeled = len(X_un)
nb_samples = nb_labeled + nb_unlabeled

n_bands = X.shape[-1]

X_t = np.concatenate((X, X_un))
Y_t = np.concatenate((Y, -1*np.ones(nb_unlabeled)))

w = np.random.uniform(-0.1, 0.1, size=n_bands)
eta_labeled = np.random.uniform(0.0, 0.1, size=nb_labeled)
eta_unlabeled = np.random.uniform(0.0, 0.1, size=nb_unlabeled)
y_unlabeled = np.random.uniform(-1.0, 1.0, size=nb_unlabeled)
b = np.random.uniform(-0.1, 0.1, size=1)

C_labeled = 1.0
C_unlabeled = 10.0

theta0 = np.hstack((w, eta_labeled, eta_unlabeled, y_unlabeled, b))

In [36]:
nb_labeled

4656

Define objective function

In [46]:
def svm_target(theta, Xd, Yd):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    s_eta_labeled = np.sum(theta[n_bands:n_bands+nb_labeled])
    s_eta_unlabeled = np.sum(theta[n_bands+nb_labeled:n_bands+nb_samples])
    return (C_labeled*s_eta_labeled) + (C_unlabeled*s_eta_unlabeled) + (0.5*np.dot(wt.T, wt))

Labeled and unlabeled constraints

In [47]:
def labeled_constraint(theta, Xd, Yd, idx):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    c = Yd[idx] * (np.dot(Xd[idx], wt) + theta[-1]) + \
    theta[n_bands:n_bands+nb_labeled][idx] - 1.0
    return (c>=0)[0]

def unlabeled_constraint(theta, Xd, idx):
    wt = theta[0:n_bands].reshape((n_bands, 1))
    c = theta[n_bands+nb_labeled:n_bands+nb_samples+nb_unlabeled][idx-nb_samples+nb_unlabeled]*\
    (np.dot(Xd[idx],wt) + theta[-1]) + \
    theta[n_bands+nb_labeled:n_bands+nb_samples][idx-nb_samples+nb_unlabeled] - 1.0
    return (c>=0)[0]

Slack constraints

In [48]:
def eta_labeled_constraint(theta, idx):
    return theta[n_bands:n_bands+nb_labeled][idx] >= 0

def eta_unlabeled_constraint(theta, idx):
    return theta[n_bands+nb_labeled:n_bands+nb_samples][idx-nb_samples+nb_unlabeled] >= 0

def y_constraint(theta, idx):
    return np.power(theta[n_bands+nb_samples:n_bands+nb_samples+nb_unlabeled][idx], 2) == 1.0

SVM constraints for scipy

In [49]:
svm_constraints = []

for i in range(nb_labeled):
    svm_constraints.append({
        'type': 'ineq',
        'fun': labeled_constraint,
        'args': (X, Y, i)
    })
    svm_constraints.append({
        'type': 'ineq',
        'fun': eta_labeled_constraint,
        'args': (i, )
    })
    
for i in range(nb_labeled, nb_samples):
    svm_constraints.append({
        'type': 'ineq',
        'fun': unlabeled_constraint,
        'args': (X, i)
    })
    svm_constraints.append({
        'type': 'ineq',
        'fun': eta_unlabeled_constraint,
        'args': (i, )
    })
    
for i in range(nb_unlabeled):
    svm_constraints.append({
        'type': 'eq',
        'fun': y_constraint,
        'args': (i,)
    })

Run scipy to optimze this

In [50]:
result=minimize(fun=svm_target, 
               x0=theta0,
               constraints=svm_constraints,
               args=(X_t,Y_t),
               method='SLSQP',
               tol=0.0001,
               options={'maxiter': 1000}
               )

IndexError: index 4656 is out of bounds for axis 0 with size 4656

In [43]:
theta0[n_bands+nb_labeled:n_bands-nb_samples]

array([], dtype=float64)

To be able to perform multiclass classification we need to create n_classes amount of TSVM's and train each on classifying if a sample belongs to one class or not. These will then need to perform a winner-takes-it-all approach to choose which class a sample belongs to. We also need to transform the linear kernel to a gaussian kernel. It is also benefitial to instead move to Lagrangian optimization.

## Run results

In [92]:
run_results = utils.metrics(prediction, test_gt, ignored_labels=ignored_labels, n_classes=n_classes)

mask = np.zeros(test_gt.shape, dtype='bool')
for l in ignored_labels:
    mask[test_gt == l] = True
prediction += 1
prediction[mask] = 0

color_prediction = convert_to_color(prediction)
utils.display_predictions(color_prediction, vis, gt=convert_to_color(test_gt), caption="Prediction vs. test ground truth")

utils.show_results(run_results, vis, label_values=label_values)

Confusion matrix :
[[1625  162    0    0    0    0    0    0    0    0    0    0    0    0
     0    0]
 [  10 3216    0    0    0    0    0    4    0    0    0    0   12  114
     0    0]
 [   0    0    0    0 1031    0    0    0   22  908   13    2    0    0
     0    0]
 [   0    0    0    0    8    0    0   10    0    0    0    0    0    0
    11 1365]
 [   0    0    0    0 2301    0    0    1    0    5    1    0    0    0
     0  219]
 [   0  912    0    0    1    0    0   12    2  643   18    0    6 2365
     0    0]
 [ 177    0    0    0    1    0    0 2795    0    0    0    0    0    0
   606    0]
 [   0    0    0    0    0    0    0 7526    1  151    1    0    6    9
  2160    6]
 [   0    0    0    0    0    0    0    1 4488  728  764    0    2    0
     0    0]
 [   0    0    0    0   11    0    0   34   34 2285   25    1    3    5
    23   20]
 [   0    0    0    0    3    0    0    0   69  108  783    5    0    0
     0    0]
 [   0    0    0    0    1    0    0    8    0

/home/oscar/anaconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
